# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto
## Part III: Neighbourhood Cluster Explore

### Introduction

This notebook explores boroughs and neighbourhoods in Toronto. The borough and neighbourhood information is obtained from pre-processed dataframe from previous sections of the project. Here, we explores the neighbourhoods of downtown Toronto, which locates at the heart of the city. Top venues of interests located within 500 M radius of each neighbourhood are obtained from queries by Foursquare API. Based on that, all neighbourhoods in downtown Toronto are clustered according to the types of venues near them and the results are visualized on the map of Toronto with markers of the neighbourhood in downtown Toronto.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Import relevant libraries</a>

2. <a href="#item2">Create a map of Toronto with neighbourhoods superimposed on top</a>

3. <a href="#item3">Explore the neighbourhoods in North York</a>

4. <a href="#item4">Explore and cluster neighbourhoods in Downtown Toronto</a>

</font>
</div>

### 1. Import relevant libraries

In [127]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### 2. Create a map of Toronto with neighbourhoods superimposed on top.

First, we read the csv file containing Toronto neighbourhood information from data pre-process in previous steps.

In [128]:
file_name = 'Toronto_Neighbourhood_and_Coordinates.csv'
df = pd.read_csv(file_name)
df.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


Then, we summarize and show simple statistics of neighbourhoods in Toronto.

In [129]:
# Examine the loaded data for simple statisics 
print('There are {} boroughs and {} neighbourhoods in Toronto, ON.'.format(len(df['Borough'].unique()),len(df['Neighbourhood'].unique())))

There are 11 boroughs and 103 neighbourhoods in Toronto, ON.


Now, we acquire geographical coordinates of the city of Toronto using geolocator, and plot a map centered at the city.
Notice that neighbourhoods in the city is marked as blue circles on top of the map.

In [130]:
# Get geographical coordinates of Toronto, ON
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [133]:
# map of Toronto
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### 3. Explore the neighbourhoods in North York

Here, we choose one borough in Toronto, that is North York, and explore the first neighbourhood, Parkwoods, of the borough by searching for top 100 venues within 800 M radius from the center of the neighbourhood.

First, we get the geographical coordinates of North York from the loaded neighbourhood information of Toronto.

In [135]:
# Find the first neighbourhood in North York
df_nyork = df[df['Borough'] == 'North York']

neighbourhood = df_nyork.loc[0,'Neighbourhood']
neigh_lat = df_nyork.loc[0,'Latitude']
neigh_lon = df_nyork.loc[0, 'Longitude']

print("The geographical coordinates of {} are {}, {}.".format(neighbourhood, neigh_lat, neigh_lon))

The geographical coordinates of Parkwoods are 43.7532586, -79.3296565.


Then, we save the client credentials of Foursquare API and use its get() command to obtain top venues near Parkwoods.

In [136]:
CLIENT_ID = 'DRPWJPB40IVVKQBT41MULEGTN4WG5SJO4GDCG5XUPXJPRJHS' # your Foursquare ID
CLIENT_SECRET = 'OKEP4LQIQTJZL1K1CM4LHPLJ5X2JO2WQ4I1D1VLI0RLJTRIZ'  # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DRPWJPB40IVVKQBT41MULEGTN4WG5SJO4GDCG5XUPXJPRJHS
CLIENT_SECRET:OKEP4LQIQTJZL1K1CM4LHPLJ5X2JO2WQ4I1D1VLI0RLJTRIZ


In [137]:
radius = 800
LIMIT =  100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neigh_lat, neigh_lon, VERSION, radius, LIMIT)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cb78a236a60715adc99232d'},
 'response': {'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 6,
  'suggestedBounds': {'ne': {'lat': 43.76045860720001,
    'lng': -79.31970728375885},
   'sw': {'lat': 43.7460585928, 'lng': -79.33960571624115}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.751976046055574,
          'lng': -79.33214044722958}],
        'distance': 245,
        'cc': 'CA',

Based on the results we got from get() query, we perform data process to extract key information such as name, location, and type of each venue we found, and save them in a new dataframe for further analysis.

In [138]:
results = results['response']['groups'][0]['items']
nearby_venues = json_normalize(results)
nearby_venues.head()

,reasons.count,reasons.items,referralId,venue.categories,venue.id,venue.location.address,venue.location.cc,venue.location.city,venue.location.country,venue.location.crossStreet,venue.location.distance,venue.location.formattedAddress,venue.location.labeledLatLngs,venue.location.lat,venue.location.lng,venue.location.postalCode,venue.location.state,venue.name,venue.photos.count,venue.photos.groups
0,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4e8d9dcdd5fbbbb6b3003c7b-0,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",4e8d9dcdd5fbbbb6b3003c7b,Toronto,CA,Toronto,Canada,NaN,245,"[Toronto, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.75197604605557...",43.751976,-79.332140,NaN,ON,Brookbanks Park,0,[]
1,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4cb11e2075ebb60cd1c4caad-1,"[{'id': '4bf58dd8d48988d1f9941735', 'name': 'F...",4cb11e2075ebb60cd1c4caad,29 Valley Woods Road,CA,Toronto,Canada,NaN,312,"[29 Valley Woods Road, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.75197441585782...",43.751974,-79.333114,NaN,ON,Variety Store,0,[]
2,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4ebdd5a077c885a64d8b3b07-2,"[{'id': '4bf58dd8d48988d1f9931735', 'name': 'R...",4ebdd5a077c885a64d8b3b07,Don Valley Parkway,CA,Toronto,Canada,at York Mills Rd.,722,"[Don Valley Parkway (at York Mills Rd.), Toron...","[{'label': 'display', 'lat': 43.75889876504828...",43.758899,-79.334099,NaN,ON,DVP at York Mills,0,[]
3,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-5b3262dc0d8a0f002ced33ad-3,"[{'id': '4bf58dd8d48988d1f8941735', 'name': 'F...",5b3262dc0d8a0f002ced33ad,205 Cassandra Blvd,CA,North York,Canada,NaN,729,"[205 Cassandra Blvd, North York ON M3A 1V2, Ca...","[{'label': 'display', 'lat': 43.75392761, 'lng...",43.753928,-79.320635,M3A 1V2,ON,S&A Tile & Flooring Services,0,[]
4,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4e45173c7d8bee2823a4e857-4,"[{'id': '52f2ab2ebcbc57f1066b8b4f', 'name': 'B...",4e45173c7d8bee2823a4e857,Sandivet,CA,Toronto,Canada,York Mills,741,"[Sandivet (York Mills), Toronto ON, Canada]","[{'label': 'display', 'lat': 43.75965484896959...",43.759655,-79.332223,NaN,ON,TTC Stop #09083,0,[]


In [139]:
# a simple function to retrieve venue category from the query results
def get_category(row):
    category = row['venue.categories'][0]['name']
    return category

In [140]:
# we filter out useful information and save it in a new dataframe
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_fil =nearby_venues.loc[:, filtered_columns]
nearby_venues_fil['venue.categories'] = nearby_venues_fil.apply(get_category, axis = 1)
nearby_venues_fil

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114
2,DVP at York Mills,Road,43.758899,-79.334099
3,S&A Tile & Flooring Services,Furniture / Home Store,43.753928,-79.320635
4,TTC Stop #09083,Bus Stop,43.759655,-79.332223
5,TTC Stop 9083,Bus Stop,43.759251,-79.334000


Now, let's summarize the findings and print out all types of venues near Parkwoods along with the number of the venues in each category.

In [141]:
venue_grp = nearby_venues_fil.groupby('venue.categories').count()
venue_grp

,venue.name,venue.location.lat,venue.location.lng
venue.categories,,,
Bus Stop,2,2,2
Food & Drink Shop,1,1,1
Furniture / Home Store,1,1,1
Park,1,1,1
Road,1,1,1


In [142]:
# summary of venues near Parkwoods
print("There are: ")
for i in range(venue_grp.shape[0]):
    print('   {} {}'.format(venue_grp.iloc[i,0], venue_grp.index.values[i]))

print('within {} M radius of {}.'.format(radius, neighbourhood))

There are: 
   2 Bus Stop
   1 Food & Drink Shop
   1 Furniture / Home Store
   1 Park
   1 Road
within 800 M radius of Parkwoods.


**Observations: from the nearby venues we got within 800 M radius of Parkwoods, North York, we found only a few venues of interests. This may suggest that the Parkwoods might not be an attractive neighbourhood to live in North York. So we want to further explore other places in Toronto.**

### 4. Explore and cluster neighbourhoods in Downtown Toronto 

In this part of the notebook, we explore neighbourhoods in Downtown Toronto in a similar way to the work in the previous part. 

First, we define a function that gets input of neighbourhood information, such as name, location, from a pre-processed dataframe, and returns top 100 venues near each of the input neighbourhood. The function will be used later when we obtain a dataframe that contains the neighbourhood information of Downtown Toronto.

In [96]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Then, We want to obtain a dataframe which only contains all rows of 'Downtown Toronto'.

In [97]:
dt_toronto = df[df['Borough'] == 'Downtown Toronto'].reset_index(drop = True)
dt_toronto.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


Now it's time to use the function "getNearbyVenues" we defined earlier to get top venues near each neighbourhood. The results are saved in a new dataframe.

In [143]:
# get the top 100 venues within 500 M of each of the neighbourhood in Downtown Toronto
names = dt_toronto['Neighbourhood']
latitudes = dt_toronto['Latitude']
longitudes = dt_toronto['Longitude']

dt_toronto_venues = getNearbyVenues(names, latitudes, longitudes, radius=500)

Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
First Canadian Place, Underground city
Church and Wellesley


Before further process, let's first check the basics of the new dataframe.

In [144]:
dt_toronto_venues.head()
print(dt_toronto_venues.shape)

(1290, 7)


Now, we want to process the venue data so that it can be used for clustering process.

First, we encode the venue category using onehot encoding.

In [145]:
# one hot encoding
dt_toronto_onehot = pd.get_dummies(dt_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dt_toronto_onehot['Neighbourhood'] = dt_toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [dt_toronto_onehot.columns[-1]] + dt_toronto_onehot.columns.to_list()[:-1]
dt_toronto_onehot = dt_toronto_onehot[fixed_columns]

dt_toronto_onehot.head()

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

Then, we want to group the data by neighbourhood and get the mean value of the encoded venue category. We want to save the results in a new feature dataframe for clustering process.

In [146]:
dt_toronto_features = dt_toronto_onehot.groupby('Neighbourhood').mean().reset_index()

In [147]:
dt_toronto_features = dt_toronto_features.drop('Neighbourhood', axis = 1)
dt_toronto_features.head(20)

,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.00,0.010000,0.010000,0.000000,0.03,0.0000,0.000000,0.0000,0.000000,0.030000,0.000000,0.030000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.01,0.020000,0.000000,0.000000,0.01,0.030000,0.010000,0.000000,0.040000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.060000,0.000000,0.000000,0.00,0.01,0.000000,0.000000,0.020000,0.000000,0.020000,0.000000,0.010000,0.010000,0.010000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.01,0.00,0.000000,0.01,0.000000,0.010000,0.000000,0.020000,0.010000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.030000,0.00,0.010000,0.010000,0.000000,0.000000,0.010000,0.020000,0.010000,0.000000,0.000000,0.000000,0.00,0.01,0.00,0.000000,0.010000,0.000000,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0

Now, we want to set up the K-Means model for unsupervised learning process. Here, we use 5 clusters and each neighbourhood in Downtown Toronto is labeled as one of the five clusters.

In [148]:
# set number of clusters
kclusters = 5

# run k-means clustering
kmeans = KMeans(n_clusters = kclusters, random_state=0).fit(dt_toronto_features)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 4, 0, 0, 3, 2, 0, 0, 0], dtype=int32)

We also want to add the label information from of the KMean model to the Downtown Toronto neighbourhood dataframe as a new column.

In [149]:
dt_toronto['Label'] = kmeans.labels_
dt_toronto

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Label
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,0
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,4
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0
5,M6G,Downtown Toronto,Christie,43.669542,-79.422564,3
6,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568,2
7,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752,0
8,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576,0
9,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817,0


Finally, we produce a map of Downtown Toronto with markers showing all neighbourhoods in different colours, each for one cluster.

In [169]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dt_toronto['Latitude'], dt_toronto['Longitude'], dt_toronto['Neighbourhood'], dt_toronto['Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

We can also examine each cluster and print it out below:

In [171]:
for label in range(kclusters):
    print("Cluster {}".format(label))
    print(dt_toronto[dt_toronto['Label'] == label])    

Cluster 0
   PostalCode           Borough  \
0         M5A  Downtown Toronto   
1         M5B  Downtown Toronto   
3         M5E  Downtown Toronto   
4         M5G  Downtown Toronto   
7         M5J  Downtown Toronto   
8         M5K  Downtown Toronto   
9         M5L  Downtown Toronto   
10        M5S  Downtown Toronto   
12        M5V  Downtown Toronto   
13        M4W  Downtown Toronto   
15        M4X  Downtown Toronto   
16        M5X  Downtown Toronto   
17        M4Y  Downtown Toronto   

                                        Neighbourhood   Latitude  Longitude  \
0                           Harbourfront, Regent Park  43.654260 -79.360636   
1                            Ryerson, Garden District  43.657162 -79.378937   
3                                         Berczy Park  43.644771 -79.373306   
4                                  Central Bay Street  43.657952 -79.387383   
7   Harbourfront East, Toronto Islands, Union Station  43.640816 -79.381752   
8            Design Excha